In [0]:
file_location="/FileStore/tables/tips.csv"
file_type="csv"

df=spark.read.csv(file_location,header=True,inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560325168603410|   Sun2959|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478071379779230|   Sun4608|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011812112971322|   Sun4458|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           11.84|  Nathaniel Harris|4676137647685994|   Sun5260|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|            6.15|      Tonya Carter|4832732618637221|   Sun2251|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|            6.32|        Erik Smith| 213140353657882|   S

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)
 |-- price_per_person: double (nullable = true)
 |-- Payer Name: string (nullable = true)
 |-- CC Number: long (nullable = true)
 |-- Payment ID: string (nullable = true)



In [0]:
df.columns

Out[4]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'price_per_person',
 'Payer Name',
 'CC Number',
 'Payment ID']

In [0]:
### handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [0]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+
|total_bill| tip|   sex|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|sex_indexed|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560325168603410|   Sun2959|        1.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478071379779230|   Sun4608|        0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011812112971322|   Sun4458|        0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|           11.84|  Nathaniel Harris|4676137647685994|   Sun5260|        0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|            6.15|      Tonya Carter|4832732618637221|   Sun2251|        1.0|
|     25

In [0]:
indexer=StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["sex_indexed","smoker_indexed","day_indexed","time_indexed"])
df_r=indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560325168603410|   Sun2959|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478071379779230|   Sun4608|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis Walters|6011812112971322|   Sun4458|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.show()

+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|price_per_person|        Payer Name|       CC Number|Payment ID|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+----------------+------------------+----------------+----------+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|            8.49|Christy Cunningham|3560325168603410|   Sun2959|        1.0|           0.0|        1.0|         0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|            3.45|    Douglas Tucker|4478071379779230|   Sun4608|        0.0|           0.0|        1.0|         0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             7.0|    Travis 

In [0]:
output.select('Independent Features').show()

+--------------------+
|Independent Features|
+--------------------+
|[1.01,2.0,1.0,0.0...|
|[1.66,3.0,0.0,0.0...|
|[3.5,3.0,0.0,0.0,...|
|[3.31,2.0,0.0,0.0...|
|[3.61,4.0,1.0,0.0...|
|[4.71,4.0,0.0,0.0...|
|[2.0,2.0,0.0,0.0,...|
|[3.12,4.0,0.0,0.0...|
|[1.96,2.0,0.0,0.0...|
|[3.23,2.0,0.0,0.0...|
|[1.71,2.0,0.0,0.0...|
|[5.0,4.0,1.0,0.0,...|
|[1.57,2.0,0.0,0.0...|
|[3.0,4.0,0.0,0.0,...|
|[3.02,2.0,1.0,0.0...|
|[3.92,2.0,0.0,0.0...|
|[1.67,3.0,1.0,0.0...|
|[3.71,3.0,0.0,0.0...|
|[3.5,3.0,1.0,0.0,...|
|(6,[0,1],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
finalized_data=output.select("Independent Features","total_bill")

In [0]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
|[1.71,2.0,0.0,0.0...|     10.27|
|[5.0,4.0,1.0,0.0,...|     35.26|
|[1.57,2.0,0.0,0.0...|     15.42|
|[3.0,4.0,0.0,0.0,...|     18.43|
|[3.02,2.0,1.0,0.0...|     14.83|
|[3.92,2.0,0.0,0.0...|     21.58|
|[1.67,3.0,1.0,0.0...|     10.33|
|[3.71,3.0,0.0,0.0...|     16.29|
|[3.5,3.0,1.0,0.0,...|     16.97|
|(6,[0,1],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[19]: DenseVector([3.2714, 3.4977, -0.8433, 2.2801, -0.6681, -0.2974])

In [0]:
regressor.intercept

Out[20]: 1.1980829841788454

In [0]:
pred_results=regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,1],[1.25,2.0])|     10.51|12.282742892370475|
| (6,[0,1],[2.0,2.0])|     12.69|14.736321117640887|
| (6,[0,1],[2.0,2.0])|     13.37|14.736321117640887|
|(6,[0,1],[2.72,2.0])|     13.28|17.091756213900485|
|(6,[0,1],[3.15,3.0])|     20.08|21.996155829425994|
|(6,[0,1],[3.27,2.0])|     17.78| 18.89104691243212|
|(6,[0,1],[3.39,2.0])|     11.61|19.283619428475387|
|[1.01,2.0,1.0,0.0...|     16.99| 9.986194766633247|
|[1.25,2.0,1.0,0.0...|      8.51| 9.805827437574258|
|[1.36,3.0,1.0,0.0...|     18.64|13.663367010317724|
|[1.5,2.0,0.0,0.0,...|     12.46|11.096202919002007|
|[1.5,2.0,1.0,0.0,...|      8.35|10.623686845997732|
|[1.76,2.0,0.0,1.0...|     11.24|16.231258592642366|
|[1.8,2.0,1.0,0.0,...|     12.43|11.605118136105897|
|[2.0,2.0,0.0,0.0,...|     13.13| 14.06818799037691|
|[2.0,2.0,0.0,1.0,...|     13.81|  17.01640362

In [0]:
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[25]: (0.43239105957961177, 4.067844717363938, 30.28833360884794)